In [10]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
# from feature_extractor import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
# from cuml.svm import SVC

In [11]:
train_df = pd.read_csv("../emnist/emnist-balanced-train.csv", header=None)
test_df = pd.read_csv("../emnist/emnist-balanced-test.csv", header=None)

# Add test dataset into train dataset
df = pd.concat([train_df, test_df], ignore_index=True)
df.head()

del train_df
del test_df

In [12]:

#Reading the file
label_map = pd.read_csv("../emnist/emnist-balanced-mapping.txt",
                        delimiter = ' ',
                        index_col=0,
                        header=None)
label_map = label_map.iloc[:, 0]


In [13]:
#Initialising an empty dictionary
label_dictionary = {}

#Running a loop for ASCII equivalent to character conversion
for index, label in enumerate(label_map):
    label_dictionary[index] = chr(label)

print(label_dictionary)


{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R', 28: 'S', 29: 'T', 30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z', 36: 'a', 37: 'b', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'n', 44: 'q', 45: 'r', 46: 't'}


In [14]:
print(df.shape)
print(df.describe())

(131600, 785)
                 0         1         2         3         4              5    \
count  131600.000000  131600.0  131600.0  131600.0  131600.0  131600.000000   
mean       23.000000       0.0       0.0       0.0       0.0       0.002036   
std        13.564712       0.0       0.0       0.0       0.0       0.295477   
min         0.000000       0.0       0.0       0.0       0.0       0.000000   
25%        11.000000       0.0       0.0       0.0       0.0       0.000000   
50%        23.000000       0.0       0.0       0.0       0.0       0.000000   
75%        35.000000       0.0       0.0       0.0       0.0       0.000000   
max        46.000000       0.0       0.0       0.0       0.0      63.000000   

                 6              7              8              9    ...  \
count  131600.000000  131600.000000  131600.000000  131600.000000  ...   
mean        0.012728       0.019354       0.016284       0.017447  ...   
std         1.312534       1.586618       1.521459  

In [15]:
df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,43,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131595,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131596,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131597,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131598,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_nums = df[df[0] <= 9]
df_lletres = df[df[0] >= 10]

In [17]:
X_nums = df_nums.iloc[:,1:]
y_nums = df_nums.iloc[:,0]
train_x_nums, test_x_nums, train_y_nums, test_y_nums = train_test_split(X_nums, y_nums, test_size=0.2, random_state=42, shuffle=True)
X_lletres = df_lletres.iloc[:,1:]
y_lletres = df_lletres.iloc[:,0]
train_x_lletres, test_x_lletres, train_y_lletres, test_y_lletres = train_test_split(X_lletres, y_lletres, test_size=0.2, random_state=42, shuffle=True)

In [18]:
resh = lambda x: np.array(x).reshape(28, 28)
train_x_nums_2d = train_x_nums.apply(resh, axis=1)
test_x_nums_2d = test_x_nums.apply(resh, axis=1)
train_x_lletres_2d = train_x_lletres.apply(resh, axis=1)
test_x_lletres_2d = test_x_lletres.apply(resh, axis=1)

In [24]:
print(train_x_nums.shape)
print(train_x_lletres.shape)

(22400, 784)
(82880, 784)


In [21]:
SVClassifier_nums_1d = SVC(kernel='linear', verbose=True, random_state=42, C=1.0)
SVClassifier_nums_1d.fit(train_x_nums, train_y_nums)


[LibSVM]

SVC(kernel='linear', random_state=42, verbose=True)

In [22]:
SVC_pred_nums_1d = SVClassifier_nums_1d.predict(test_x_nums)
print(classification_report(test_y_nums, SVC_pred_nums_1d))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       536
           1       0.95      0.99      0.97       578
           2       0.89      0.93      0.91       534
           3       0.89      0.92      0.91       565
           4       0.90      0.92      0.91       546
           5       0.90      0.86      0.88       595
           6       0.97      0.95      0.96       574
           7       0.94      0.95      0.94       565
           8       0.94      0.85      0.89       556
           9       0.90      0.89      0.89       551

    accuracy                           0.92      5600
   macro avg       0.92      0.92      0.92      5600
weighted avg       0.92      0.92      0.92      5600



In [25]:
SVClassifier_lletres_1d = SVC(kernel='linear', verbose=True, random_state=42, C=1.0)
SVClassifier_lletres_1d.fit(train_x_lletres, train_y_lletres)

[LibSVM]

In [ ]:
SVC_pred_lletres_1d = SVClassifier_lletres_1d.predict(test_x_lletres)
print(classification_report(test_y_lletres, SVC_pred_lletres_1d))